Zastosowane 4 metody dra Kontka dla danych z 2015 roku (Duda & Komorowski)

In [1]:
import pandas as pd
import re
import os
import numpy as np
from scipy.stats import median_abs_deviation

from src.utilities import presidential_data
from src.utilities import kontek_clustering as clustering
from src.utilities import kontek_methods as methods

### load data

In [25]:
year = "2015"
ext = "xls"
df_2015_r1 = presidential_data.get_df(year, "1", ext)


df_2015_r2 = presidential_data.load_data(year, "2", ext)
df_2015_r2 = df_2015_r2.rename(columns={
    "Teryt Gminy": "teryt_gmina",
    "Numer obwodu": "polling_station_id"
})

# In 2015 round 2 data there is no postal code, so we need to join it to round 1
df_2015_r2 = df_2015_r2.merge(
    df_2015_r1[["teryt_gmina", "polling_station_id", "postal_code"]],
    on=["teryt_gmina", "polling_station_id"],
    how="left"
)
df_2015_r2 = presidential_data.process_df(df_2015_r2, year)

------------

### IMPLEMENTACJA METOD ZAPROPONOWANYCH PRZEZ DR KONTKA

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5296441

2.3. Wykrywanie wartości odstających

Główna innowacja analityczna niniejszego badania polega na oszacowaniu potencjalnego wpływu
anormalnych komisji wyborczych na poziomie ogólnokrajowym. Aby to osiągnąć, w pierwszej
kolejności zidentyfikowano wartości odstające w czterech kategoriach nieprawidłowości:

In [50]:
cand_A = "komorowski"
cand_B = "duda"

df_2015 = presidential_data.join_both_rounds(cand_A, cand_B, df_2015_r1, df_2015_r2)

In [54]:
# Add buckets
df_2015 = clustering.add_bialek_postal_buckets(df_2015)

### 1. Nadmierne poparcie dla Andrzeja Dudy (względem mediany w ramach lokalnej grupy)

In [56]:
df = df_2015.copy()

2.4 Przeliczenie wyników

Aby uwzględnić niepewność i wrażliwość zastosowanego podejścia, obliczenia przeprowadzono dla
trzech różnych progów detekcji wartości odstających: k > 2.0, k > 2.5 oraz k > 3.0, gdzie k oznacza
liczbę jednostek odchylenia bezwzględnego od mediany (MAD) względem mediany w grupie
lokalnej. Wyższe wartości k wyodrębniają jedynie najbardziej skrajne przypadki, zapewniając tym
samym konserwatywną estymację potencjalnego wpływu. Jednocześnie jednak ograniczają zdolność
metody do wychwytywania mniejszych, lecz wciąż istotnych odchyleń.

In [57]:
# Let's set k = 2
k = 2

In [58]:
df = methods.add_anomaly_1(df, cand_A, cand_B, k=k, new_col_name="anomaly_1")

In [59]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_1")

k > 2.0
komorowski: 3206
duda: 2928
---
k > 2.5
komorowski: 2298
duda: 2038
---
k > 3.0
komorowski: 1718
duda: 1464
---


Wyniki:

Dla k=2, takich komisji, w których "za duże" poparcie ma Duda jest 2928 a Komorowski 3206

2025: 3750 i 4551 

2020: 2893 i 3430


In [60]:
# drop k_score columns
# should be done in add_anomaly func, but to show different anomaly results based on k

df = df.drop(columns=[
    f"{cand_A}_k_score_1",
    f"{cand_B}_k_score_1"
])

### 2. Nadmierny względny wzrost poparcia dla Andrzeja Dudy między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla Bronislawa Komorowskiego w tej samej grupie lokalnej;

za JB: 

Nie podano wprost jak to było obliczone więc kolejno:

  1. Dla danego kandydata obliczam względny wzrost między pierwszą a drugą turą (dzieląc wynik z drugiej przez wynik z pierwszej)
  2. Następnie odnoszę go do wzrostu drugiego kandydata - liczę różnicę między względnymi wzrostami.
  3. Dalej tak jak w pierwszym typie anomalii - dla tych różnic liczę medianę grupy, MAD grupy oraz odchylenie k w komisji.

In [61]:
df = methods.add_anomaly_2(df, cand_A, cand_B, k=k, new_col_name="anomaly_2")

In [62]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_2")

k > 2.0
komorowski: 2754
duda: 3074
---
k > 2.5
komorowski: 1950
duda: 2262
---
k > 3.0
komorowski: 1464
duda: 1707
---


k > 2.0

2025:
trzaskowski: 3552
nawrocki: 3127

2020:
trzaskowski: 3795
duda: 1863



In [63]:
df = df.drop(columns=[
    f"{cand_A}_k_score_2",
    f"{cand_B}_k_score_2"
])

### 3. Komisje, w których Duda uzyskał więcej głosów niż Komorowski w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę Komorowskiego;

  1. Sprawdzamy, w których grupach dany kandydat miał większą medianę
  2. Sumujemy komisje, w których wygrał kandydat A mimo, że większą medianę miał kandydat B i na odwrót.

In [64]:
df = methods.add_anomaly_3(df, cand_A, cand_B, k=2, new_col_name="anomaly_3")

In [65]:
# na korzyść kandydat A, czyli większą medianę miał B, a więcej głosów dostał A.
cand_A, sum(df['higher_median_' + cand_B] & (df[cand_A + '_r2'] > df[cand_B + '_r2']))

('komorowski', 1806)

vs 2025:

('trzaskowski', 2608)

2020:

('trzaskowski', 1885)

In [66]:
cand_B, sum(df['higher_median_' + cand_A] & (df[cand_B + '_r2'] > df[cand_A + '_r2']))

('duda', 1750)

vs 2025:

('nawrocki', 1843)

2020:

('duda', 1999)

**WYNIKI**:

W grupach, w których większą medianę miał Duda, było 1750 komisji, w których wyższy wynik uzyskał Trzaskowski.

W grupach, w których większą medianę miał Trzaskowski, było 1806 komisji, w których wyższy wyniki uzyskał Nawrocki.


In [67]:
df = df.drop(columns=[
    f"higher_median_{cand_A}",
    f"higher_median_{cand_B}"	
])

### 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [68]:
cand_A, sum(df[cand_A + '_r2']<df[cand_A + '_r1'])

('komorowski', 286)

vs 2025:

('trzaskowski', 128)

2020:

('trzaskowski', 106)

W 286 komisjach Komorowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

In [69]:
cand_B, sum(df[cand_B + '_r2']<df[cand_B + '_r1'])

('duda', 93)

vs 2025:

('nawrocki', 112)

vs 2020:

('duda', 284)

W 93 komisjach Duda uzyskał mniej głosów w drugiej turze niż w pierwszej.


Przykładowe anomalie na korzyść Komorowskiego:

In [70]:
df[df[cand_B + '_r2'] < df[cand_B + '_r1']].sort_values(by=cand_B + '_r1', ascending=False).head()

,teryt_gmina,polling_station_id,komorowski_r1,duda_r1,postal_code,komorowski_r2,duda_r2,postal_clean,bucket,komorowski_anomaly_1,duda_anomaly_1,komorowski_anomaly_2,duda_anomaly_2,komorowski_anomaly_3,duda_anomaly_3
1486,22605,1,71,114,59-516,196,112,59516,2316,False,False,True,False,True,False
12758,143406,4,88,104,05-280,169,101,05280,76,False,False,True,False,True,False
1273,22308,2,0,94,55-010,0,93,55010,1310,False,False,False,False,False,True
4773,61408,15,58,52,24-150,57,46,24150,484,False,False,False,False,True,False
26199,302903,22,7,51,64-200,7,50,64200,1515,False,False,False,False,False,True


In [71]:
# Anomalie na korzysc
df[cand_A + '_anomaly_4'] = df[cand_B + '_r2']<df[cand_B + '_r1']
df[cand_B + '_anomaly_4'] = df[cand_A + '_r2']<df[cand_A + '_r1']

To są rzeczywiście bardzo podejrzane przypadki i o takich przypadkach powinniśmy alarmować w pierwszej kolejności. Po pierwsze, ju na etapie wprowadzania do systemu, a po drugie do ewentualnej kontroli i ponownego liczenia glosów

## Sumowanie anomalii

### Na korzyść Komorowskiego

In [72]:
df[cand_A + '_sum_anomalies'] = df[[
    cand_A + '_anomaly_1', 
    cand_A + '_anomaly_2',
    cand_A + '_anomaly_3',
    cand_A + '_anomaly_4']].sum(axis=1)

In [73]:
for number_of_anomalies in [1,2,3,4]:
    print(f"{number_of_anomalies} anomalies:")
    print(cand_A, sum(df[cand_A + '_sum_anomalies']>=number_of_anomalies))

1 anomalies:
komorowski 7073
2 anomalies:
komorowski 756
3 anomalies:
komorowski 30
4 anomalies:
komorowski 0


vs 2025:

1 anomalies:
trzaskowski 8161


2 anomalies:
trzaskowski 1179


3 anomalies:
trzaskowski 34


4 anomalies:
trzaskowski 3

2020:

1 anomalies:
trzaskowski 8406


2 anomalies:
trzaskowski 962


3 anomalies:
trzaskowski 24


4 anomalies:
trzaskowski 2

In [75]:
# Nie ma komisji z wszystkimi 4 anomaliami
df[df[cand_A + '_sum_anomalies']>=4]

,teryt_gmina,polling_station_id,komorowski_r1,duda_r1,postal_code,komorowski_r2,duda_r2,postal_clean,bucket,komorowski_anomaly_1,duda_anomaly_1,komorowski_anomaly_2,duda_anomaly_2,komorowski_anomaly_3,duda_anomaly_3,komorowski_anomaly_4,duda_anomaly_4,komorowski_sum_anomalies


### Na korzyść Dudy

In [76]:
df[cand_B + '_sum_anomalies'] = df[[
    cand_B + '_anomaly_1', 
    cand_B + '_anomaly_2',
    cand_B + '_anomaly_3',
    cand_B + '_anomaly_4']].sum(axis=1)

In [77]:
for number_of_anomalies in [1,2,3,4]:
    print(f"{number_of_anomalies} anomalies:")
    print(cand_B, sum(df[cand_B + '_sum_anomalies']>=number_of_anomalies))

1 anomalies:
duda 7124
2 anomalies:
duda 842
3 anomalies:
duda 71
4 anomalies:
duda 1


vs 2025:

1 anomalies:
nawrocki 6871


2 anomalies:
nawrocki 483


3 anomalies:
nawrocki 34


4 anomalies:
nawrocki 2

vs 2020:

1 anomalies:
duda 6243


2 anomalies:
duda 571


3 anomalies:
duda 47


4 anomalies:
duda 0



In [79]:
# 1 komisja ze wszystkimi 4 anomaliami
df[df[cand_B + '_sum_anomalies']>=4]

,teryt_gmina,polling_station_id,komorowski_r1,duda_r1,postal_code,komorowski_r2,duda_r2,postal_clean,bucket,komorowski_anomaly_1,duda_anomaly_1,komorowski_anomaly_2,duda_anomaly_2,komorowski_anomaly_3,duda_anomaly_3,komorowski_anomaly_4,duda_anomaly_4,komorowski_sum_anomalies,duda_sum_anomalies
21096,246201,66,392,181,41-902,358,569,41902,1087,False,True,False,True,False,True,False,True,0,4
